In [12]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('2022_2023_football_summer_transfers_v2.csv')

In [15]:
# change column 'position' from 19 values to 4

Defender=['Centre-Back','Right-Back','Left-Back','Defender','defence']
Forward = ['Right Winger','Centre-Forward','Left Winger','Forward','Sweeper','Second Striker','attack']
Midfielder = ['Central Midfield', 'Attacking Midfield', 'Defensive Midfield', 'Left Midfield','Right Midfield',
              'Midfielder']
conditions = [
    df['position'].isin(Defender),
    df['position'].isin(Forward),
    df['position'].isin(Midfielder),
    df['position']=='Goalkeeper']

values = ['Defender', 'Forward','Midfielder','Goalkeeper']

df['position'] = np.select(conditions, values)

# Exploratory Data Analysis


In [16]:
# get df with top 100 transfers by fee
df_top_100 = df.nlargest(100, 'fee')

# new column rand with method min: lowest rank in the group
df_top_100['rank100'] = df_top_100.fee.rank(method='min', ascending=False).astype('int')
fig = go.Figure(data=[go.Table(
    header=dict(values=list(['<b>Rank</b>', "<b>Player's name</b>", '<b>League left</b>',
                             '<b>Club left</b>', '<b>League joined</b>', '<b>Club joined</b>',
                             '<b>Fee, mln EUR</b>']),
                fill_color='light blue',
                align='left'),
    cells=dict(values=[df_top_100.rank100, df_top_100.name, df_top_100.league_from, df_top_100.club_from,
                       df_top_100.league_to, df_top_100.club_to, df_top_100.fee],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(
        title=dict(
            text="<b>Top 100 transfers by fee</b>",
            x=0.05,
            y=0.92,
            font=dict(
                family="Arial",
                size=20
            )
        ),
        margin = dict(t=100, l=50, r=50, b=40),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)'
)

fig.show()

In [17]:
import plotly.express as px

# Copy and rename columns
top_100_labels = df_top_100.copy()
top_100_labels = top_100_labels.rename(columns={'country_to': 'Country', 'league_to': 'League',
                                                'club_to': 'Team', 'fee': 'Fee, mln EUR'})

# Create sunburst chart
fig = px.sunburst(top_100_labels, path=['Country', 'League', 'Team', 'name'], values='Fee, mln EUR',
                  color='Country', hover_data=['Fee, mln EUR'],
                  color_discrete_map={'England':'#3366CC', 'Italy':'#DC3912', 'France':'#109618',
                                      'Spain':'#990099', 'Germany':'#DD4477'})

# Update layout
fig.update_layout(
    title=dict(
        text="<b>Top 100 transfers by fee, Interact chart</b>",
        x=0.05,
        y=0.92,
        font=dict(
            family="Arial",
            size=20
        )
    ),
    margin=dict(t=0, l=0, r=0, b=0),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)'
)

fig.show()


In [18]:
df_legue_to = df.groupby(['league_to', 'country_to'], as_index=False) \
                    .agg({'fee': sum}).round(2) \
                    .rename(columns={'fee': 'Spend', 'league_to': 'League', 'country_to': 'Country'})

df_legue_from = df.groupby(['league_from', 'country_from'], as_index=False) \
                    .agg({'fee': sum}).round(2) \
                    .rename(columns={'fee': 'Income', 'league_from': 'League', 'country_from': 'Country'})

df_league_all = df_legue_to.merge(df_legue_from, on=['Country', 'League'])
df_league_all['Net spend'] = round(df_league_all.Income - df_league_all.Spend, 2)
df_league_top5 = df_league_all.nlargest(5, 'Spend').copy()

labels = df_league_top5.League
set_colors = ['#3366CC', '#DC3912', '#109618', '#990099', '#DD4477']
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=df_league_top5.Spend, name="mln EUR", opacity=0.85,
                     marker_colors=set_colors), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=df_league_top5.Income, name="mln EUR", opacity=0.85),1, 2)
# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.35, hoverinfo="value+name", textposition='inside', textinfo='percent+label')
fig.update_layout(
        title=dict(
            text='<b>Transfer spend and income for top leagues</b>',
            x=0.5,
            y=0.92,
            font=dict(
                family="Arial",
                size=20
            )
        ),
        font=dict(
            family="Verdana",
            size=14
        ),
        margin = dict(t=100, l=50, r=50, b=40),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=False,
        # Add annotations in the center of the donut pies.
        annotations=[dict(text='<b>Spend</b>', x=0.19, y=0.53, font_size=17, showarrow=False),
                     dict(text='<b>Income</b>', x=0.82, y=0.53, font_size=17, showarrow=False),
                     dict(text='<b>4.6 bn EUR</b>', x=0.175, y=0.44, font_size=14, showarrow=False),
                     dict(text='<b>3.3 bn EUR</b>', x=0.83, y=0.44, font_size=14, showarrow=False)])
fig.show()

Record after record: Premier League invests as much as the rest of the top leagues combined

In [19]:
league_top5_for_melt = df_league_top5.drop(columns='Country').copy()
df_league_unpiv = league_top5_for_melt.melt(id_vars='League', var_name='Spend') \
                                    .sort_values('value', ascending=False) \
                                    .rename(columns={'Spend': 'Balance', 'value': 'Fee, mln EUR'})
league_dict = dict(League=["Premier League", "Serie A", "Ligue 1", "LaLiga", "Bundesliga"])

fig = px.bar(df_league_unpiv, x="Fee, mln EUR", y="League", color="League",
             orientation='h', range_x=[-1350,2300],
             animation_frame="Balance",
             animation_group="Balance",
             opacity=0.7, text_auto='.2s',
             color_discrete_sequence=set_colors, # color of bars
             category_orders=league_dict)
fig.update_layout(
        title=dict(
            text='<b>Transfer balance for top leagues</b>',
            x=0.5,
            y=0.92,
            font=dict(
                family="Arial",
                size=20
            )
        ),
        xaxis_title="<b>Fee, mln EUR</b>",
        yaxis_title="",
        yaxis=dict(
            showticklabels=False
        ),
        xaxis=dict(
            dtick=1000
        ),
        margin = dict(t=100, l=50, r=50, b=40),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        legend_title="<b>League</b>"
)
fig.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()

In [20]:
top_leagues = ('Serie A', 'Premier League', 'LaLiga', 'Ligue 1', 'Bundesliga')
df_from_to_top5 = df.query('league_from in @top_leagues & league_to in @top_leagues')
league_top = df_from_to_top5 \
                    .groupby(['league_to', 'league_from'], as_index=False) \
                    .agg({'fee': 'sum'}) \
                    .sort_values('fee', ascending=False) \
                    .rename(columns={'league_from': 'League left', 'fee': 'Fee, mln EUR',
                                     'league_to': 'League joined'})
# to work correctly animation_frame add 1 row from LaLiga to Bundesliga
league_top.loc[len(league_top.index)] = ['Bundesliga', 'LaLiga', 1]

fig = px.bar(league_top, x="Fee, mln EUR", y="League joined", color="League joined",
             orientation='h', range_x=[0,650],
             animation_frame="League left", animation_group="League left",
             opacity=0.7, text_auto='.3s',
             color_discrete_sequence=set_colors, # color of bars
             category_orders=league_dict)
fig.update_layout(
        title=dict(
            text='<b>Transfers between top leagues</b>',
            x=0.5,
            y=0.92,
            font=dict(
                family="Arial",
                size=20
            )
        ),
        xaxis_title="<b>Fee, mln EUR</b>",
        yaxis_title="<b>League joined</b>",
        yaxis=dict(
            showticklabels=False
        ),
        xaxis=dict(
            showgrid=False,
            zeroline=False,
            showticklabels=False
        ),
        margin = dict(t=100, l=80, r=50, b=40),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        legend_title="<b>League joined</b>"
)
fig.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5, textangle=0)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()

In [21]:
country_list = ('Italy', 'England', 'Spain', 'Germany', 'France')
league_list = ('Serie A', 'Premier League', 'LaLiga', 'Ligue 1', 'Bundesliga')
top_league_to = df.query('country_to in @country_list & league_to in @league_list & fee > 0 & loan == False').copy()

top_league_to = top_league_to.rename(columns={'league_to': 'League',
                                                'club_to': 'Team', 'fee': 'Fee, mln EUR'})
fig = px.sunburst(top_league_to, path=['League', 'Team', 'name'], values='Fee, mln EUR',
                  color='Fee, mln EUR',
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(top_league_to['Fee, mln EUR'],
                                                       weights=top_league_to['Fee, mln EUR']))

fig.update_layout(
    grid= dict(columns=2, rows=1),
    margin = dict(t=0, l=0, r=0, b=0),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)'
)

fig.show()

**Popular Player and Position**

In [22]:
tl_position = top_league_to.copy()
tl_position = tl_position.rename(columns={'position': 'Position', 'name': 'Name'})

fig = px.box(tl_position, x="Fee, mln EUR", y="League", color='League',
             hover_data=["Team", "Name"],
             animation_frame="Position",
             animation_group="Position",
             points="all",
             color_discrete_sequence=set_colors, # color of bars
             category_orders=league_dict
            )

fig.update_layout(
    title=dict(
        text='<b>Transfer fee (median and quantiles) to top leagues by player position</b>',
        x=0.5,
        y=0.92,
        font=dict(
            family="Arial",
            size=20
        )
    ),
    xaxis_title="<b>Fee, mln EUR</b>",
    yaxis_title="<b>League</b>",
    yaxis=dict(
        showticklabels=False
    ),
    xaxis=dict(
        dtick=10,
        gridwidth=1,
        zerolinewidth=2,
    ),
    margin = dict(t=100, l=80, r=50, b=40),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    font=dict(
        family="Verdana",
        size=12,
    ),
    legend_title="<b>League</b>"
)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()

In [23]:
top_league_to.age = top_league_to.age.astype('int')
tl_hist = top_league_to.rename(columns={'age': 'Age'}).copy()

fig = px.histogram(tl_hist, x="Age", facet_col='League', color='League',
             opacity=0.6, nbins=20,
             color_discrete_sequence=set_colors, # color of bars
             category_orders=league_dict) # league order

fig.update_layout(
    title=dict(
        text='<b>Number of transfers to top leagues by player age</b>',
        x=0.5,
        y=0.92,
        font=dict(
            family="Arial",
            size=20
        )
    ),
    yaxis_title="<b>Number of transfers</b>",
    margin = dict(t=100, l=100, r=50, b=70),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)'
)
fig.show()

In [24]:
fig = px.histogram(tl_hist, x="Age", y="Fee, mln EUR", histfunc="avg", facet_col='League', color='League',
             opacity=0.6, nbins=20,
             color_discrete_sequence=set_colors, # color of bars
             category_orders=league_dict)

fig.update_layout(
    title=dict(
        text='<b>Average of transfers fee to top leagues by player age</b>',
        x=0.5,
        y=0.92,
        font=dict(
            family="Arial",
            size=20
        )
    ),
    yaxis_title="<b>Average fee, mln EUR</b>",
    margin = dict(t=100, l=100, r=50, b=70),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)'
)
fig.show()


In this dataset it is difficult to see the prevalence of the same age of players in terms of average fee. Robert Lewandowski is one player, who joined LaLiga at 33 years old, that's why it is anomaly. And we see again that Premier League has gone far ahead

In [26]:
df_club_to = df.groupby(['club_to', 'league_to'], as_index=False) \
                    .agg({'fee': sum}).round(2) \
                    .rename(columns={'fee': 'Spend', 'club_to': 'Team', 'league_to': 'League'})

df_club_from = df.groupby(['club_from', 'league_from'], as_index=False) \
                    .agg({'fee': sum}).round(2) \
                    .rename(columns={'fee': 'Income', 'club_from': 'Team', 'league_from': 'League'})

df_club_all = df_club_to.merge(df_club_from, how='outer', on=['League', 'Team'])
df_club_all['Net spend'] = round(df_club_all.Income - df_club_all.Spend, 2)
df_club_top50 = df_club_all.nlargest(50, 'Spend').copy()

fig = go.Figure(data=[go.Table(
    header=dict(values=list(["<b>League</b>", "<b>Team</b>", '<b>Spend, mln EUR</b>',
                             '<b>Income, mln EUR</b>', '<b>Net spend, mln EUR</b>']),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_club_top50.League, df_club_top50.Team, df_club_top50.Spend,
                       df_club_top50.Income, df_club_top50['Net spend']],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(
        title=dict(
            text="<b>Transfer balance for the teams</b>",
            x=0.05,
            y=0.92,
            font=dict(
                family="Arial",
                size=20
            )
        ),
        margin = dict(t=100, l=50, r=50, b=40),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)'
)

fig.show()

In [27]:
df_club_top20 = df_club_all.nlargest(20, 'Spend').copy()
labels = df_club_top20.Team
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=df_club_top20.Spend, name="mln EUR", opacity=0.85,
                     marker_colors=set_colors), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=df_club_top20.Income, name="mln EUR", opacity=0.85,
                     marker_colors=set_colors),1, 2)
# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.35, hoverinfo="value+name", textposition='inside', textinfo='percent+label')
fig.update_layout(
        title=dict(
            text='<b>Transfer spend and income for top 20 teams by spend</b>',
            x=0.5,
            y=0.92,
            font=dict(
                family="Arial",
                size=20
            )
        ),
        margin = dict(t=100, l=50, r=50, b=40),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=False,
        # Add annotations in the center of the donut pies.
        annotations=[dict(text='<b>Spend</b>', x=0.19, y=0.53, font_size=17, showarrow=False),
                     dict(text='<b>Income</b>', x=0.82, y=0.53, font_size=17, showarrow=False),
                     dict(text='<b>2.8 bn EUR</b>', x=0.175, y=0.44, font_size=14, showarrow=False),
                     dict(text='<b>1.4 bn EUR</b>', x=0.83, y=0.44, font_size=14, showarrow=False)])
fig.show()


In [28]:
df_club_top20 = df_club_all.nlargest(20, 'Spend').copy()
club_top20_for_melt = df_club_top20.drop(columns='League').copy()
df_club_unpiv = club_top20_for_melt.melt(id_vars='Team', var_name='Spend') \
                                    .rename(columns={'Spend': 'Balance', 'value': 'Fee, mln EUR'})

fig = px.bar(df_club_unpiv, x="Fee, mln EUR", y="Team", color="Team",
             orientation='h', range_x=[-230,300],
             animation_frame="Balance",
             animation_group="Balance",
             opacity=0.7,
             color_discrete_sequence=px.colors.qualitative.D3 + px.colors.qualitative.T10)
fig.update_layout(
        title=dict(
            text='<b>Transfer balance for top 20 teams by spend</b>',
            x=0.5,
            y=0.92,
            font=dict(
                family="Arial",
                size=20
            )
        ),
        xaxis_title="<b>Fee, mln EUR</b>",
        yaxis_title="",
        showlegend=False,
        font=dict(
            family="Verdana",
            size=12
        ),
        margin = dict(t=100, l=50, r=0, b=40),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)'
)
#fig.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()

In [48]:
tl_to_fee_diff = df.query('fee > 0 & loan == False').copy()
tl_to_fee_diff['fee_diff'] = round(tl_to_fee_diff.fee - tl_to_fee_diff.market_value, 2)
tl_to_fee_diff_top10 = tl_to_fee_diff.nlargest(10, 'fee_diff').copy()
# new column rand with method min: lowest rank in the group
tl_to_fee_diff_top10['rank10'] = tl_to_fee_diff_top10.fee_diff.rank(
                                                    method='min', ascending=False).astype('int')

fig = go.Figure(data=[go.Table(
    header=dict(values=list(['<b>Rank</b>', "<b>Player's name</b>", '<b>Team left</b>', '<b>Team joined</b>',
                             '<b>Market value, mln EUR</b>', '<b>Fee, mln EUR</b>', '<b>Fee diff, mln EUR</b>']),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tl_to_fee_diff_top10.rank10, tl_to_fee_diff_top10.name, tl_to_fee_diff_top10.club_from,
                       tl_to_fee_diff_top10.club_to, tl_to_fee_diff_top10.market_value, tl_to_fee_diff_top10.fee,
                       tl_to_fee_diff_top10.fee_diff],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(
        title=dict(
            text="<b>Top 10 overpaid transfers (transfer fee > market value)</b>",
            x=0.05,
            y=0.92,
            font=dict(
                family="Arial",
                size=20
            )
        ),
        margin = dict(t=100, l=50, r=50, b=40),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)'
)

fig.show()


In [49]:
tl_fee_diff_antitop10 = tl_to_fee_diff.nsmallest(10, 'fee_diff').copy()
# new column rand with method min: lowest rank in the group
tl_fee_diff_antitop10['rank10'] = tl_fee_diff_antitop10.fee_diff.rank(method='min').astype('int')
fig = go.Figure(data=[go.Table(
    header=dict(values=list(['<b>Rank</b>', "<b>Player's name</b>", '<b>Team left</b>', '<b>Team joined</b>',
                             '<b>Market value, mln EUR</b>', '<b>Fee, mln EUR</b>', '<b>Fee diff, mln EUR</b>']),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tl_fee_diff_antitop10.rank10, tl_fee_diff_antitop10.name, tl_fee_diff_antitop10.club_from,
                       tl_fee_diff_antitop10.club_to, tl_fee_diff_antitop10.market_value,
                       tl_fee_diff_antitop10.fee, tl_fee_diff_antitop10.fee_diff],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(
        title=dict(
            text="<b>Top 10 underpaid transfers (market value > transfer fee)</b>",
            x=0.05,
            y=0.92,
            font=dict(
                family="Arial",
                size=20
            )
        ),
        margin = dict(t=100, l=50, r=50, b=40),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)'
)

fig.show()

# Focus on Juventus FC


In [47]:
# Filter data for players who left Juventus FC
df_left_juventus = df[df['club_from'] == 'Juventus FC']

# Create a table figure
fig = go.Figure(data=[go.Table(
    header=dict(values=['<b>Player\'s name</b>','<b>Age</b>', '<b>League left</b>',
                        '<b>Club left</b>', '<b>League joined</b>', '<b>Club joined</b>',
                        '<b>Fee, mln EUR</b>'],
                fill_color='light blue',
                align='left'),
    cells=dict(values=[df_left_juventus['name'],df_left_juventus['age'], df_left_juventus['league_from'], df_left_juventus['club_from'],
                       df_left_juventus['league_to'], df_left_juventus['club_to'], df_left_juventus['fee']],
               fill_color='lavender',
               align='left'))
])

# Update layout
fig.update_layout(
    title=dict(
        text="<b>Players who left Juventus FC</b>",
        x=0.05,
        y=0.92,
        font=dict(
            family="Arial",
            size=20
        )
    ),
    margin=dict(t=100, l=50, r=50, b=40),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)'
)

fig.show()


In [45]:
import plotly.graph_objs as go

# Filter data for players who joined Juventus FC
df_joined_juventus = df[df['club_to'] == 'Juventus FC']

# Create a table figure
fig = go.Figure(data=[go.Table(
    header=dict(values=['<b>Player\'s name</b>', '<b>Age</b>', '<b>League joined</b>',
                        '<b>Club joined</b>', '<b>League left</b>', '<b>Club left</b>',
                        '<b>Fee, mln EUR</b>'],
                fill_color='light blue',
                align='left'),
    cells=dict(values=[df_joined_juventus['name'], df_joined_juventus['age'],
                       df_joined_juventus['league_to'], df_joined_juventus['club_to'],
                       df_joined_juventus['league_from'], df_joined_juventus['club_from'],
                       df_joined_juventus['fee']],
               fill_color='lavender',
               align='left'))
])

# Update layout
fig.update_layout(
    title=dict(
        text="<b>Players who joined Juventus FC</b>",
        x=0.05,
        y=0.92,
        font=dict(
            family="Arial",
            size=20
        )
    ),
    margin=dict(t=100, l=50, r=50, b=40),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)'
)

fig.show()


In [42]:
import plotly.graph_objs as go
import pandas as pd

# Convert the 'age' column to numeric, handling any errors by coercing them to NaN
df['age'] = pd.to_numeric(df['age'], errors='coerce')

# Filter data for Juventus transfers
juventus_left = df[df['club_from'] == 'Juventus FC']
juventus_joined = df[df['club_to'] == 'Juventus FC']

# Calculate average ages
avg_age_left = juventus_left['age'].mean()
avg_age_joined = juventus_joined['age'].mean()

# Create a bar graph
fig = go.Figure(data=[
    go.Bar(name='Left Juventus', x=['Left Juventus'], y=[avg_age_left], marker_color='indianred'),
    go.Bar(name='Joined Juventus', x=['Joined Juventus'], y=[avg_age_joined], marker_color='lightseagreen')
])

# Update layout
fig.update_layout(
    title="<b>Average Age of Players Who Joined and Left Juventus FC</b>",
    xaxis_title="Transfer Type",
    yaxis_title="Average Age",
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig.show()


This strategy suggests that Juventus prioritizes investing in experienced players, likely reflecting a desire for players who are competition-ready and can immediately contribute at a high level. Rather than focusing on younger talents who may require more development.

In [43]:
# Ensure the 'fee' column is numeric
df['fee'] = pd.to_numeric(df['fee'], errors='coerce')

# Filter data for players who joined Juventus
juventus_joined = df[df['club_to'] == 'Juventus FC']

# Calculate the average transfer fee by position
avg_fee_by_position = juventus_joined.groupby('position')['fee'].mean().dropna()

# Create a horizontal bar graph
fig = go.Figure(data=[
    go.Bar(
        x=avg_fee_by_position.values,
        y=avg_fee_by_position.index,
        orientation='h',
        marker_color='royalblue'
    )
])

# Update layout
fig.update_layout(
    title="<b>Average Transfer Fee by Position for Players Joining Juventus FC</b>",
    xaxis_title="Average Transfer Fee (mln EUR)",
    yaxis_title="Position",
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)'
)

fig.show()

In [44]:
import plotly.express as px
import pandas as pd

# Ensure the 'age' column is numeric
df['age'] = pd.to_numeric(df['age'], errors='coerce')

# Filter data for players who joined Juventus
juventus_joined = df[df['club_to'] == 'Juventus FC']

# Create a box plot for age distribution by position
fig = px.box(
    juventus_joined,
    x='position',
    y='age',
    color='position',
    title="Age Distribution by Position for Players Joining Juventus FC",
    labels={'age': 'Age', 'position': 'Position'},
    template='plotly_white'
)

# Update layout
fig.update_layout(
    xaxis_title="Position",
    yaxis_title="Age",
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig.show()


Juventus prefers to invest in younger defenders, this could indicate a strategic approach centered on developing a robust, long-term defensive line.
